This notebook goes over all of the major components of this project:
1. Simulation: Can be thought of as a black box that takes in a solution and outputs coverage metrics. Used to evaluate solutions and generate a dataset.
2. Machine learning: A multilayer perceptron (MLP) is trained to predict coverage metrics given a solution.
3. Optimization: The MLP is embedded within a MIP which attempts to find the solution that the MLP predicts will have the highest coverage.

In [1]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from ems_data import EMSData
from simulation import Simulation
from multilabel_mlp import MultilabelMLP
from mip_models import *

Before doing anything, we need to load the data. The EMSData class reads and preprocesses the data for usage in simulation and MIP models.

In [2]:
# Skip this cell if you already have ems_data.pkl and don't need to regenerate with different parameters
ems_data = EMSData()
ems_data.save_instance('ems_data.pkl')

In [3]:
ems_data = EMSData.load_instance('ems_data.pkl')

n_stations = len(ems_data.stations)
n_demand_nodes = len(ems_data.demand_nodes)
demand = ems_data.demand_nodes.demand
print(f'# stations: {n_stations}')
print(f'# demand nodes: {n_demand_nodes}')

# stations: 8
# demand nodes: 62


# Simulation
The Simulation class pulls relevant data from an EMSData instance and runs the simulation. Simulations are used to evaluate a solution (i.e., the number of ambulances at each station) as well as generate a dataset for the MLP.

In [4]:
# Pickled Simulation instance is used by HTCondor jobs
sim = Simulation(ems_data)
sim.save_instance('simulation.pkl')

In [5]:
# Run multiple replications of the simulation using a solution that places 2 ambulances at each station
sim.run([2]*8)

,covered0,total0,covered1,total1,covered2,total2,covered3,total3,covered4,total4,...,covered57,total57,covered58,total58,covered59,total59,covered60,total60,covered61,total61
0,9,9,6,7,31,47,32,42,1,1,...,14,24,67,112,12,20,18,25,3,5
1,5,5,5,6,28,36,36,44,0,0,...,14,20,74,117,12,15,15,31,0,3
2,3,3,2,2,32,45,40,53,0,0,...,6,13,58,105,16,22,25,39,0,3
3,4,4,2,2,31,39,40,52,2,2,...,9,14,69,107,15,26,21,35,1,1
4,3,4,2,2,28,38,36,49,3,3,...,11,18,59,108,8,18,14,30,0,1
5,2,2,6,7,31,37,39,50,3,3,...,10,15,65,108,14,22,16,30,1,5
6,4,4,5,5,18,30,40,52,2,2,...,11,12,50,94,14,20,13,24,1,2
7,5,5,2,3,33,44,35,46,1,1,...,11,18,57,101,10,21,22,37,0,3
8,3,3,4,5,29,37,35,41,0,0,...,9,16,56,97,12,18,19,30,1,3
9,1,2,3,3,28,41,44,59,1,1,...,10,14,63,99,18,32,15,31,1,3


In [6]:
# Run multiple replications and evaluate coverage
def evaluate_solution(solution):
    sim_result = sim.run(solution)
    result = sim_result.sum()  # Sum over replications
    result = np.array([result[f'covered{i}']/result[f'total{i}'] for i in range(n_demand_nodes)])
    result = np.nan_to_num(result, nan=0.0)
    coverage = demand@result / demand.sum()  # Estimated long-term coverage
    return coverage

evaluate_solution([2]*8)

0.8317462853398422

The dataset was generated using HTCondor. `n_jobs = 100` jobs are run, each job performs the simulation for `solutions_per_job = 5000` solutions, and `n_replications = 10` replications are ran per solution.

Steps:
1. Run `htcondor_setup.py` to generate the `settings<Process>.csv` files. These files contain the solutions to be simulated on each HTCondor job. Note that the script assumes `simulation.pkl` has already been created.
2. Move the following files to the HTCondor submit server:
    - `simulation.sub`
    - `run_job.sh`
    - `run_job.py`
    - `simulation.py`
    - `simulation.pkl`
    - `settings$(Process).csv` for each `Process`
    
    Then run `condor_submit simulation.sub`.
3. Once the jobs are done, move the `results<Process>.csv` files to a new folder named `sim_results`. Then run `create_dataset.py` to create the dataset from the `results<Process>.csv` files. For each solution, the script sums the `covered<i>` and `total<i>` columns over the replications and defines `coverage<i>` as the ratio of the two sums. The resulting dataset has columns `station<i>` for `i` in `range(n_stations)`, and `coverage<i>` for `i` in `range(n_demand_nodes)`. The dataset is saved as `dataset.csv`.

In [7]:
dataset = pd.read_csv('dataset.csv')
dataset

,station0,station1,station2,station3,station4,station5,station6,station7,coverage0,coverage1,...,coverage52,coverage53,coverage54,coverage55,coverage56,coverage57,coverage58,coverage59,coverage60,coverage61
0,17,0,0,5,3,2,3,0,0.685714,0.722222,...,0.900000,0.725869,0.740864,0.746797,0.754967,0.654088,0.611754,0.633484,0.517751,0.380952
1,3,0,6,1,4,9,4,3,0.717949,0.608696,...,0.666667,0.730994,0.759259,0.833333,0.866242,0.660606,0.662931,0.742489,0.558405,0.652174
2,3,2,1,9,6,3,0,6,0.940000,0.936170,...,0.655172,0.711240,0.744604,0.792117,0.801205,0.658228,0.637857,0.693277,0.535256,0.424242
3,5,5,0,3,1,3,6,7,0.928571,0.913793,...,0.709677,0.705545,0.728555,0.776692,0.812903,0.618182,0.621827,0.727700,0.537705,0.408163
4,1,5,1,11,3,3,4,2,0.931818,0.943396,...,0.692308,0.739754,0.721154,0.792131,0.805556,0.635802,0.648528,0.688372,0.599359,0.510638
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,3,0,3,5,7,3,2,7,0.804878,0.677966,...,0.666667,0.726115,0.722531,0.792593,0.808642,0.691275,0.650709,0.647321,0.543478,0.714286
499996,1,4,1,4,3,0,4,13,0.875000,0.931034,...,0.750000,0.660194,0.579805,0.498184,0.455621,0.621622,0.520524,0.478814,0.436364,0.416667
499997,0,3,0,1,1,2,11,12,0.864865,0.892857,...,0.608696,0.654344,0.669653,0.692937,0.734940,0.674847,0.615044,0.552846,0.500000,0.571429
499998,3,0,7,6,4,0,4,6,0.605263,0.700000,...,0.739130,0.680435,0.605206,0.540364,0.435714,0.506173,0.547982,0.439462,0.483974,0.555556


# Machine Learning
The MLP takes as input the solution and outputs the coverage probabilities for each demand node.

In [8]:
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

NVIDIA GeForce GTX 1070


device(type='cuda')

In [9]:
# Move dataset into tensors compatible with model and split into train/dev/test sets
X = dataset[[f'station{i}' for i in range(n_stations)]].to_numpy()
y = dataset[[f'coverage{i}' for i in range(n_demand_nodes)]].to_numpy()
X = torch.tensor(X, dtype=torch.float32, device=device)
y = torch.tensor(y, dtype=torch.float32, device=device)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size=20000)
X_train, X_dev, y_train, y_dev = train_test_split(X_train, y_train, random_state=0, train_size=0.5)
X_train.shape, X_dev.shape, X_test.shape, y_train.shape, y_dev.shape, y_test.shape

(torch.Size([10000, 8]),
 torch.Size([10000, 8]),
 torch.Size([480000, 8]),
 torch.Size([10000, 62]),
 torch.Size([10000, 62]),
 torch.Size([480000, 62]))

In [10]:
# Find best performing training sample
sim_scores = y_train.cpu().numpy()@demand / demand.sum()
print(f'Long-term coverage of best training sample: {sim_scores.max()}')

Long-term coverage of best training sample: 0.9372910108223353


In [11]:
# Train MLP
mlp = MultilabelMLP(n_stations, [200], n_demand_nodes, dropout=0.1).to(device)
init_train_loss = mlp.evaluate_model(X_train, y_train)
init_dev_loss = mlp.evaluate_model(X_dev, y_dev)
print(f'Initial train loss: {init_train_loss}, initial dev loss: {init_dev_loss}')
mlp.train_model(X_train, y_train, X_dev, y_dev, batch_size=32, max_epochs=10, tolerance=0.0, verbose=True)

Evaluating:   0%|          | 0/79 [00:00<?, ?it/s]

Evaluating: 100%|██████████| 79/79 [00:00<00:00, 858.72it/s]


Initial train loss: 0.8782253237816595, initial dev loss: 0.8776612513388357


Evaluating: 100%|██████████| 313/313 [00:00<00:00, 1788.60it/s]


Avg train loss: 0.4944803964138031, dev loss: 0.4716763605917654


...

Avg train loss: 0.46935626096725463, dev loss: 0.4677628218127835


Evaluating: 100%|██████████| 313/313 [00:00<00:00, 1213.19it/s]


Avg train loss: 0.46922708678245545, dev loss: 0.4677486151910597


Evaluating: 100%|██████████| 313/313 [00:00<00:00, 1199.24it/s]

Avg train loss: 0.46914475455284116, dev loss: 0.4677888174241589


In [12]:
# Example of loading a saved model
mlp = MultilabelMLP.load_model('model.pt').to(device)
mlp

MultilabelMLP(
  (0): Linear(in_features=8, out_features=200, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.0, inplace=False)
  (3): Linear(in_features=200, out_features=62, bias=True)
)

In [13]:
# Predict on test set and evaluate some metrics
y_hat = mlp.predict(X_test)
abs_diff = np.absolute(y_hat.cpu().numpy() - y_test.cpu().numpy())
abs_diff.mean(), np.median(abs_diff), abs_diff.max(), (abs_diff > 0.5).mean()

Predicting: 100%|██████████| 3750/3750 [00:01<00:00, 1875.00it/s]


(0.0229693, 0.013706818, 0.9361497, 8.635752688172043e-06)

# Optimization
The trained MLP is embedded within a MIP which attempts to find the solution that the MLP predicts will have the highest coverage.

In [14]:
# Daskin's MEXCLP model
coverage = compute_coverage(ems_data)
solution = mexclp(demand=demand, coverage=coverage, n_ambulances=30, busy_fraction=0.5, verbose=True)
print(f'MEXCLP solution: {solution}')
score = evaluate_solution(solution)
print(f'Long-term coverage of MEXCLP with p=0.5: {score}')

Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-17


Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: AMD Ryzen 5 3600 6-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 63 rows, 1868 columns and 2012 nonzeros
Model fingerprint: 0xe94c8367
Variable types: 0 continuous, 1868 integer (1860 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [9e-10, 4e+02]
  Bounds range     [1e+00, 3e+01]
  RHS range        [3e+01, 3e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 3 rows and 92 columns
Presolve time: 0.02s
Presolved: 60 rows, 1776 columns, 1902 nonzeros
Variable types: 0 continuous, 1776 integer (1770 binary)
Found heuristic solution: objective 378.9999996

Root relaxation: objective 7.264092e+03, 160 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    Best

In [15]:
# Our model with MLP embedded
solution = mexclp_mlp(demand=demand, mlp=mlp, n_ambulances=30, time_limit=60, verbose=True)
print(f"Our model's solution: {solution}")
score = evaluate_solution(solution)
print(f'Long-term coverage of our model: {score}')

Set parameter TimeLimit to value 60


Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: AMD Ryzen 5 3600 6-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 1557 rows, 740 columns and 17054 nonzeros
Model fingerprint: 0x96b3b266
Variable types: 532 continuous, 208 integer (200 binary)
Coefficient statistics:
  Matrix range     [5e-06, 4e+01]
  Objective range  [1e+00, 8e+02]
  Bounds range     [1e+00, 3e+01]
  RHS range        [6e-03, 4e+01]
Found heuristic solution: objective 3910.1990914
Presolve removed 49 rows and 14 columns
Presolve time: 0.06s
Presolved: 1508 rows, 726 columns, 16826 nonzeros
Variable types: 520 continuous, 206 integer (198 binary)

Root relaxation: objective 7.144664e+03, 1259 iterations, 0.05 seconds (0.10 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 71